# Data containers

## Ephemerides

In [1]:
import astropy.units as u  # for physical quantities
from sbpy.data import Ephem  # for ephemerides

eph = Ephem.from_dict({'rh': 1 * u.au, 'delta': 1 * u.au, 'phase': 60 * u.deg})
print(eph)

<QTable length=1>
   rh    delta   phase 
   AU      AU     deg  
float64 float64 float64
------- ------- -------
    1.0     1.0    60.0


In [2]:
from astropy.time import Time

epochs = {'start': Time('2022-06-22'),
          'stop': Time('2023-06-22'),
          'step': 10 * u.day}

print(epochs)

{'start': <Time object: scale='utc' format='iso' value=2022-06-22 00:00:00.000>, 'stop': <Time object: scale='utc' format='iso' value=2023-06-22 00:00:00.000>, 'step': <Quantity 10. d>}


In [3]:
eph = Ephem.from_mpc('2P',
                     epochs=epochs,    
                     location='I11')  # using Gemini South as a proxy for LSST

#  see also, `Ephem.from_horizons` and `Ephem.from_miriade`

In [4]:
eph.field_names

['Targetname',
 'Date',
 'RA',
 'Dec',
 'Delta',
 'r',
 'Elongation',
 'Phase',
 'V',
 'Proper motion',
 'Direction',
 'Azimuth',
 'Altitude',
 'Sun altitude',
 'Moon phase',
 'Moon distance',
 'Moon altitude']

In [5]:
eph.table.show_in_notebook()  # eph.table provides access to the underlying astropy table object, which has some more functionality

idx,Targetname,Date,RA,Dec,Delta,r,Elongation,Phase,V,Proper motion,Direction,Azimuth,Altitude,Sun altitude,Moon phase,Moon distance,Moon altitude
,,,deg,deg,AU,AU,deg,deg,mag,arcsec / h,deg,deg,deg,deg,,deg,deg
0,2P,2022-06-22 00:00:00.000,0.1995833333333333,2.865277777777778,3.887,3.999,89.0,14.7,23.3,9.84,49.7,143.0,-57.0,-27.0,0.41,11.0,-60.0
1,2P,2022-07-02 00:00:00.000,0.49041666666666656,3.2225,3.709,3.981,98.1,14.6,23.2,4.78,25.4,129.0,-52.0,-27.0,0.07,129.0,1.0
2,2P,2022-07-12 00:00:00.000,0.39583333333333326,3.4322222222222223,3.533,3.961,107.6,14.2,23.0,4.5,295.9,118.0,-45.0,-26.0,0.95,98.0,51.0
3,2P,2022-07-22 00:00:00.000,359.87791666666664,3.4755555555555557,3.364,3.939,117.6,13.2,22.9,10.75,266.4,110.0,-37.0,-25.0,0.36,44.0,-74.0
4,2P,2022-08-01 00:00:00.000,358.9029166666666,3.333055555555555,3.208,3.916,128.0,11.8,22.7,18.15,258.6,103.0,-28.0,-24.0,0.1,163.0,11.0
5,2P,2022-08-11 00:00:00.000,357.46208333333334,2.9919444444444445,3.068,3.891,139.0,9.8,22.5,25.63,255.1,97.0,-18.0,-23.0,0.98,56.0,39.0
6,2P,2022-08-21 00:00:00.000,355.58083333333326,2.4516666666666667,2.952,3.864,150.5,7.4,22.3,32.57,253.0,91.0,-7.0,-22.0,0.33,81.0,-84.0
7,2P,2022-08-31 00:00:00.000,353.32374999999996,1.7266666666666668,2.862,3.836,162.2,4.6,22.1,38.16,251.5,86.0,4.0,-21.0,0.14,154.0,21.0
8,2P,2022-09-10 00:00:00.000,350.8083333333333,0.8544444444444445,2.804,3.806,173.4,1.7,21.9,41.51,250.4,80.0,15.0,-20.0,1.0,14.0,27.0


## Orbits

In [6]:
from sbpy.data import Orbit

orbit = Orbit.from_dict({
    'targetname': '2P/Encke',
    'e': 0.85,
    'q': 0.34 * u.au,
    'i': 12 * u.deg,
    'Omega': 330 * u.deg,
    'w': 190 * u.deg,
    'Tp': Time('2020-06-26'),
    'epoch': Time('2021-07-05')
})
print(orbit)

<QTable length=1>
targetname    e       q       i     Omega     w               Tp                    epoch         
                      AU     deg     deg     deg                                                  
   str8    float64 float64 float64 float64 float64           Time                    Time         
---------- ------- ------- ------- ------- ------- ----------------------- -----------------------
  2P/Encke    0.85    0.34    12.0   330.0   190.0 2020-06-26 00:00:00.000 2021-07-05 00:00:00.000


In [7]:
# or, just get it from the Minor Planet Center
orbit = Orbit.from_mpc('2P')
print(orbit)

<QTable length=1>
   Q        w        Omega    desig     e       epoch   ...  number       Tp          q        P        a    
   AU      deg        deg                               ...                           AU       yr       AU   
float64  float64    float64    str1  float64     Time   ... float64      Time      float64  float64  float64 
------- --------- ----------- ----- --------- --------- ... ------- ------------- --------- ------- ---------
  4.099 186.60343 334.5166578       0.8486622 2459400.5 ...     2.0 2459026.73073 0.3355616     3.3 2.2173022


### Using openorb

Orbit transformations are (presently) performed with openorb, which requires a target name, orbit type, H, and G parameters.  The HG system doesn't always make sense for a comet, but we need to add them anyway.

Propagate the osculating elements to 2026:

In [8]:
orbit['targetname'] = '2P'
orbit['orbtype'] = 'COM'   # cometary style orbit with q, Tp (otherwise use KEP with a, M)
orbit['H'] = 15 * u.mag
orbit['G'] = 0.15

# propagate the orbit forward to 2026 with openorb
future_orbit = orbit.oo_propagate(Time('2026-06-06'))
print(future_orbit)

<QTable length=1>
 id           a                  e                 incl       ... orbtype  epoch     H       G   
              AU                                   deg        ...                   mag          
str2       float64            float64            float64      ... float64   Time  float64 float64
---- ------------------- ------------------ ----------------- ... ------- ------- ------- -------
  2P 0.33866709052606453 0.8472974903813582 11.34798709906705 ...     2.0 61197.0    15.0    0.15


Convert an orbit into an ephemeris:

In [9]:
import numpy as np

orbit = Orbit.from_horizons('Ceres')
epochs = Time('2026-01-01') + np.arange(12) * 30 * u.day
eph = Ephem.from_oo(orbit, epochs=epochs)
eph.table.show_in_notebook()

idx,targetname,RA,DEC,RA*cos(Dec)_rate,DEC_rate,alpha,elong,r,Delta,V,pa,TopEclLon,TopEclLat,OppTopEclLon,OppTopEclLat,HelEclLon,HelEclLat,OppHelEclLon,OppHelEclLat,EL,ELsun,ELmoon,lunarphase,lunarelong,x,y,z,vx,vy,vz,obsx,obsy,obsz,trueanom,epoch
,,deg,deg,deg / d,deg / d,deg,deg,AU,AU,mag,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,deg,,deg,AU,AU,AU,AU / d,AU / d,AU / d,AU,AU,AU,deg,
0,1 Ceres (A801 AA),9.675797980887554,-5.499673882448088,0.16547789421620127,0.1448084152718726,19.875754420847503,86.53888170154941,2.8870180601769735,2.774408879821769,8.863605310076032,48.811082864659596,6.705891725662259,-8.886604792209855,-93.50376508559087,-8.883141068569486,26.813021632706924,-8.538530371271305,-73.3966351785462,-8.535066647630936,90.0,90.0,90.0,0.913596633142974,61.068547401904794,2.5480568896369937,1.2878430974224153,-0.42864848463852506,-0.004866074821140865,0.00854140817113133,0.0011668725180616246,-0.17429525866271428,0.9677564243067907,-5.944542260682972e-05,232.69994313622257,2461041.500800741
1,1 Ceres (A801 AA),16.19785851757896,-0.8115762844398202,0.2615253790072533,0.16441843142274323,18.00469913998833,64.07673699972327,2.866545020947881,3.156853531366023,9.066370549644414,57.84282905919406,14.60900401793126,-7.1199090876831805,-116.14023599713431,-7.1173897819104495,32.755003873265025,-7.8461774468165535,-97.99423614180056,-7.843658141043822,90.0,90.0,90.0,0.9494908381035725,90.99889854690875,2.388171734524358,1.5364185549219538,-0.39132358430273717,-0.005787066934638915,0.008014183469288384,0.0013198623285834636,-0.6430637079023618,0.7463321894640457,-4.3317717684066004e-05,238.6242127299174,2461071.500800741
2,1 Ceres (A801 AA),25.03001231785694,4.200201622269738,0.3221189174034712,0.16727533913210976,14.000894524585945,43.98962040445116,2.844728571601356,3.4731742475740646,9.11780918584589,62.55728997482116,24.733362583154516,-5.776414216120558,-136.3142606806842,-5.775553695193512,38.76406997222905,-7.058716819075753,-122.28355329160969,-7.057856298148708,90.0,90.0,90.0,0.9739138604062573,117.78461017256889,2.201310622584286,1.7676275088046929,-0.3495785735705085,-0.006662568049697245,0.0073835798393768635,0.001461194422895181,-0.9372329458960577,0.32184405979052066,-1.4883057455922222e-05,244.6353575804441,2461101.500800741
3,1 Ceres (A801 AA),35.385182773218595,9.09923537583128,0.3602365059785513,0.15733985504760206,8.80611398024568,25.619021780040285,2.8217889992867047,3.689393176653406,9.03136918691209,66.40582399808139,36.122207308918576,-4.723497346652654,-154.79103236826867,-4.7245167028765565,44.84674141960801,-6.180436768636267,-146.06649825757924,-6.181456124860169,90.0,90.0,90.0,0.9873460347644314,141.4761615183063,1.9890080358886735,1.978395733814898,-0.3037935320061199,-0.007481028007295688,0.006651545145359813,0.0015888063406873047,-0.9810202333438985,-0.1891498273770321,1.7774910422499457e-05,250.74054063628438,2461131.500800741
4,1 Ceres (A801 AA),46.80511404810439,13.538594057424156,0.3836807432027014,0.13700265746995147,3.1466024699145616,8.76931683106714,2.797970359583463,3.789362474882859,8.761809926243718,70.34964319313217,48.1665637275187,-3.850331989356132,-172.11395431218273,-3.852933915298497,51.00985322410078,-5.216947575647167,-169.27066481560067,-5.2195495015895315,90.0,90.0,90.0,0.9936839204249175,162.11495912222478,1.7531532706537272,2.165725408848266,-0.2544114702067568,-0.008230369566501566,0.005821328909576347,0.0017005737903153717,-0.7685234602351686,-0.6513061629009601,4.574762215850403e-05,256.94634019786906,2461161.500800741
5,1 Ceres (A801 AA),58.968520990978575,17.24139619666913,0.3956548560920152,0.10872014070422363,3.3769395833190283,9.27432421113552,2.773539715709678,3.769198803254319,8.748333501008698,74.63518557607814,60.45856565539994,-3.072109286264167,171.24383864873684,-3.0756008847594605,57.260492552069884,-4.175368729151666,168.0457655454068,-4.17886032764696,90.0,90.0,90.0,0.9968950292019245,170.64429638929062,1.4960058176740487,2.326737936794598,-0.201939

## Physical properties

In [10]:
from sbpy.data import Phys
phys = Phys.from_sbdb(['Ceres', '12893', '3552'])  # JPL's Small Body Database
print(phys.field_names)

['targetname', 'H', 'H_sig', 'H_ref', 'H_note', 'G', 'G_sig', 'G_ref', 'G_note', 'diameter', 'diameter_sig', 'diameter_ref', 'diameter_note', 'extent', 'extent_sig', 'extent_ref', 'extent_note', 'GM', 'GM_sig', 'GM_ref', 'GM_note', 'density', 'density_sig', 'density_ref', 'density_note', 'rot_per', 'rot_per_sig', 'rot_per_ref', 'rot_per_note', 'pole', 'pole_sig', 'pole_ref', 'pole_note', 'albedo', 'albedo_sig', 'albedo_ref', 'albedo_note', 'BV', 'BV_sig', 'BV_ref', 'BV_note', 'UB', 'UB_sig', 'UB_ref', 'UB_note', 'spec_T', 'spec_T_sig', 'spec_T_ref', 'spec_T_note', 'spec_B', 'spec_B_sig', 'spec_B_ref', 'spec_B_note']


In [11]:
print(phys['targetname', 'H', 'D', 'albedo'])

<QTable length=3>
        targetname            H    diameter  albedo
                             mag      km           
          str26            float64 float64  float64
-------------------------- ------- -------- -------
         1 Ceres (A801 AA)    3.56    939.4    0.09
 12893 Mommert (1998 QS55)   13.98    5.214   0.179
3552 Don Quixote (1983 SA)   12.96     19.0    0.03


## Observational data

In [5]:
from sbpy.data import Obs

phot = Obs.from_mpc('24')  # get all reported data on (24) Themis
print(len(phot))

4666


In [6]:
phot[:10].table.show_in_notebook()  # just the first 10 entries

idx,number,desig,discovery,note1,note2,epoch,RA,DEC,mag,band,observatory
,,,,,,,deg,deg,mag,,
0,24,,--,--,M,2397971.38172,166.79537499999998,6.354166666666666,———,--,520
1,24,,--,--,M,2397972.37477,166.80120833333334,6.344138888888889,———,--,520
2,24,,--,--,M,2397978.42191,166.94766666666663,6.241361111111111,———,--,520
3,24,,--,--,M,2397979.38502,166.99175,6.216527777777778,———,--,520
4,24,,--,--,A,2397982.42308,167.154375,6.1257777777777775,———,--,520
5,24,,--,--,A,2397984.40804,167.28708333333333,6.058916666666667,———,--,520
6,24,,--,--,M,2397987.41103,167.52637499999997,5.937888888888889,———,--,520
7,24,,--,--,M,2399222.3727,23.836291666666664,9.566666666666666,———,--,516
8,24,,--,--,M,2399241.27806,20.344708333333333,8.260972222222222,———,--,516
